In [1]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %                   SCF-TB - PROXY APPLICATION                      %
# %                   A.M.N. Niklasson, M. Kulichenko. T1, LANL       %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# % Total Energy Function:                                            %
# % E = 2Tr[H0(D-D0)] + (1/2)*sum_i U_i q_i^2 +                       %
# %      + (1/2)sum_{i,j (i!=j)} q_i C_{ij} q_j - Efield*dipole       %
# % dipole = sum_i R_{i} q_i                                          %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

import torch
import warnings
import logging
import os
# to disable torchdynamo completely. Faster for smaller systems and single-point calculations.
# os.environ["TORCHDYNAMO_DISABLE"] = "1"  # hard-disable capture

from dftorch import (
    Constants,
    Structure,
    StructureBatch,
    MDXL,
    MDXLBatch,
    MDXLOS,
    ESDriver,
    ESDriverBatch,
)

### Configure torch and torch.compile ###
# Silence warnings and module logs
warnings.filterwarnings("ignore")
os.environ["TORCH_LOGS"] = ""  # disable PT2 logging
os.environ["TORCHINDUCTOR_VERBOSE"] = "0"
os.environ["TORCHDYNAMO_VERBOSE"] = "0"
logging.getLogger("torch.fx").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.symbolic_shapes").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.recording").setLevel(logging.CRITICAL)
# Enable dynamic shape capture for dynamo
torch._dynamo.config.capture_dynamic_output_shape_ops = True
# default data type
torch.set_default_dtype(torch.float64)

torch.cuda.empty_cache()

# MD with PBC and PME
### Note, cell dimensions must be >= (2 x cutoff) if using PME.
### Also, do not use PME without PBC.

## First, SCF and forces

In [2]:
%%time
dftorch_params = {
    "coul_method": "FULL",  # 'FULL' for full coulomb matrix, 'PME' for PME method
    "Coulomb_acc": 5e-5,  # Coulomb accuracy for full coulomb calcs or t_err for PME
    "cutoff": 10.0,  # Coulomb cutoff
    "PME_order": 4,  # Ignored for FULL coulomb method
    "SCF_MAX_ITER": 100,  # Maximum number of SCF iterations
    "SCF_TOL": 1e-6,  # SCF convergence tolerance on density matrix
    "SCF_ALPHA": 0.4,  # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.
    "KRYLOV_MAXRANK": 20,  # Maximum Krylov subspace rank
    "KRYLOV_TOL": 1e-6,  # Krylov subspace convergence tolerance in SCF
    "KRYLOV_TOL_MD": 1e-4,  # Krylov subspace convergence tolerance in MD SCF
    "KRYLOV_START": 4,  # Number of initial SCF iterations before starting Krylov acceleration
}

# Initial data, load atoms and coordinates, etc in COORD.dat
device = "cuda" if torch.cuda.is_available() else "cpu"
filename = "COORD.xyz"  # Solvated acetylacetone and glycine molecules in H20, Na, Cl
LBox = torch.tensor(
    [25.0, 25.0, 25.0], device=device
)  # Simulation box size in Angstroms. Only cubic boxes supported for now.
# Create constants container. Set path to SKF files.
const = Constants(
    filename,
    "sk_orig/mio-1-1/mio-1-1/",
    magnetic_hubbard_ldep=False,
).to(device)

# Create structure object. Define total charge and electronic temperature.
structure1 = Structure(filename, LBox, const, charge=0, Te=1000.0, device=device)

# Create ESDriver object and run SCF calculation
# electronic_rcut and repulsive_rcut are in Angstroms.
# They should be >= cutoffs defined in SKF files for the element pair with largest cutoff present in the system.
es_driver = ESDriver(
    dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device
)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const)  # Calculate forces after SCF

coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
   LMAX: 7
  Coulomb_k t 0.6 s

### Do _scf ###
  Initial dm_fermi

Starting cycle
Iter 1
Res = 0.174508520, dEc = 0.269006961, t = 0.2 s

Iter 2
Res = 0.092088242, dEc = 0.022725105, t = 0.0 s

Iter 3
Res = 0.049057698, dEc = 0.010945558, t = 0.0 s

Iter 4
Res = 0.026266178, dEc = 0.005468078, t = 0.0 s

Iter 5
  rank: 0, Fel = 0.000200
  rank: 1, Fel = 0.000013
  rank: 2, Fel = 0.000001
Res = 0.014099263, dEc = 0.005837085, t = 0.2 s

Iter 6
  rank: 0, Fel = 0.000000
Res = 0.000002478, dEc = 0.000000066, t = 0.0 s

Iter 7
  rank: 0, Fel = 0.000000
Res = 0.000000181, dEc = 0.000000001, t = 0.0 s

CPU times: user 5.1 s, sys: 619 ms, total: 5.72 s
Wall time: 3.16 s


In [3]:
# charges:
structure1.q

tensor([ 0.3670, -0.0733, -0.1296, -0.0957, -0.0733,  0.4510, -0.0760, -0.1167,
        -0.0820, -0.1783,  0.4478, -0.0762, -0.1190, -0.0816, -0.1757,  0.3878,
        -0.0824, -0.0521, -0.0561, -0.1859], device='cuda:0')

In [4]:
# MO coeffs
structure1.Q

tensor([[ 0.2977, -0.4211,  0.4329,  ..., -0.2183,  0.1995, -0.1117],
        [ 0.0684, -0.0475, -0.0528,  ..., -0.0033,  0.0567, -0.0569],
        [-0.0006,  0.0009, -0.0034,  ..., -0.0147,  0.0672, -0.0572],
        ...,
        [ 0.0610,  0.1395,  0.1946,  ...,  0.1112,  0.0836,  0.0372],
        [ 0.0675,  0.1525,  0.2084,  ...,  0.2065,  0.1583,  0.0777],
        [ 0.1524,  0.2504,  0.1775,  ...,  0.3385,  0.4118,  0.2962]],
       device='cuda:0')

In [5]:
print("Total Energy: ", structure1.e_tot)
print("Chemical Potential: ", structure1.mu0)

Total Energy:  tensor(-332.4865, device='cuda:0')
Chemical Potential:  tensor(0.3639, device='cuda:0')


## MD

In [6]:
torch.manual_seed(0)
temperature_K = torch.tensor(1.0, device=structure1.device)
mdDriver = MDXL(es_driver, const, temperature_K=temperature_K)
# Set number of steps, time step (fs), dump interval and trajectory filename
mdDriver.run(
    structure1,
    dftorch_params,
    num_steps=5,
    dt=0.3,
    dump_interval=5,
    traj_filename="md_trj.xyz",
)

########## Step = 0 ##########
coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
   LMAX: 7
  Coulomb_k t 0.4 s

H0: 0.461 s
H1: 0.149 s
  rank: 0, Fel = 0.000190
  rank: 1, Fel = 0.000012
KER: 0.139 s
F AND E: 0.165 s
ETOT = -332.48413890, EPOT = -332.59759810, EKIN = 0.00232668, T = 0.89999886, ResErr = 0.000000, t = 0.9 s
0.01145344 GB


########## Step = 1 ##########
coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
   LMAX: 7
  Coulomb_k t 0.4 s

H0: 0.464 s
H1: 0.002 s
  rank: 0, Fel = 0.000373
  rank: 1, Fel = 0.000023
KER: 0.003 s
F AND E: 0.002 s
ETOT = -332.48484726, EPOT = -332.91065033, EKIN = 0.11275084, T = 43.61391938, ResErr = 0.001167, t = 0.5 s
0.011453952 GB


########## Step = 2 ##########
coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
   LMAX: 7
  Coulomb_k t 0.4 s

H0: 0.456 s
H1: 0.002 s
  rank: 0, Fel = 0.000207
  rank: 1, Fel = 0.000011
KER: 0.003 s
F AND E: 0.001 s
ETOT = -332.48673284, EPOT = -333.39438699, EKIN = 0.42391748, T = 163.97840940, ResErr = 0.0023

# Batched MD with PBC and full Coulomb calculations.
### Systems in a batch should have same number of atoms and orbitals. Number of electrons and atom order can be arbitrary.
### Note, Batched mode does not support PME, only full Coulomb.

In [7]:
%%time
dftorch_params = {
    "coul_method": "FULL",
    "Coulomb_acc": 1e-5,  # coulomb accuracy for full coulomb calcs or t_err for PME
    "cutoff": 10.0,  # coulomb cutoff
    "PME_order": 4,
    "SCF_MAX_ITER": 20,
    "SCF_TOL": 1e-6,
    "SCF_ALPHA": 0.1,  # Linear mixing coefficient used before Krylov acceleration starts.
    "KRYLOV_MAXRANK": 20,
    "KRYLOV_TOL": 1e-6,
    "KRYLOV_TOL_MD": 1e-4,
    "KRYLOV_START": 10,
}

# Initial data, load atoms and coordinates, etc in COORD.dat
device = "cuda"

filename = ["COORD_8WATER.xyz"] * 4
LBox = torch.tensor(
    [[21, 21, 21], [22, 22, 22], [21, 21, 22], [22, 22, 21]], device=device
)

const = Constants(filename, "sk_orig/mio-1-1/mio-1-1/").to(device)

structure = StructureBatch(filename, LBox, const, charge=0, Te=1000.0, device=device)

es_driver = ESDriverBatch(
    dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device
)
es_driver(structure, const, do_scf=True, verbose=False)
es_driver.calc_forces(structure, const)

   LMAX: tensor([7, 8, 7, 8], device='cuda:0', dtype=torch.int32)

Starting cycle
Iter 1
Batch 0: Res = 7.373e-01, dEc = 4.155e+00
Batch 1: Res = 7.374e-01, dEc = 4.154e+00
Batch 2: Res = 7.373e-01, dEc = 4.154e+00
Batch 3: Res = 7.373e-01, dEc = 4.154e+00
Iter 2
Batch 0: Res = 6.415e-01, dEc = 2.899e-01
Batch 1: Res = 6.416e-01, dEc = 2.900e-01
Batch 2: Res = 6.416e-01, dEc = 2.900e-01
Batch 3: Res = 6.416e-01, dEc = 2.900e-01
Iter 3
Batch 0: Res = 5.583e-01, dEc = 2.462e-01
Batch 1: Res = 5.583e-01, dEc = 2.462e-01
Batch 2: Res = 5.583e-01, dEc = 2.462e-01
Batch 3: Res = 5.583e-01, dEc = 2.462e-01
Iter 4
Batch 0: Res = 4.859e-01, dEc = 2.096e-01
Batch 1: Res = 4.859e-01, dEc = 2.097e-01
Batch 2: Res = 4.859e-01, dEc = 2.097e-01
Batch 3: Res = 4.859e-01, dEc = 2.096e-01
Iter 5
Batch 0: Res = 4.229e-01, dEc = 1.790e-01
Batch 1: Res = 4.229e-01, dEc = 1.790e-01
Batch 2: Res = 4.229e-01, dEc = 1.790e-01
Batch 3: Res = 4.229e-01, dEc = 1.790e-01
Iter 6
Batch 0: Res = 3.681e-01, dEc = 1.53

In [8]:
torch.manual_seed(0)
temperature_K = torch.tensor([20.0, 30.0, 30.0, 30.0], device=structure.device)
mdDriver = MDXLBatch(es_driver, const, temperature_K=temperature_K)
mdDriver.run(
    structure,
    dftorch_params,
    num_steps=5,
    dt=0.3,
    dump_interval=5,
    traj_filename="batch_md_trj.xyz",
)

########## Step = 0 ##########
   LMAX: tensor([7, 8, 7, 8], device='cuda:0', dtype=torch.int32)
H0: 0.996 s
H1: 0.009 s
  rank: 0, batch 0, Fel = 0.000013
  rank: 0, batch 1, Fel = 0.000014
  rank: 0, batch 2, Fel = 0.000012
  rank: 0, batch 3, Fel = 0.000013
  Not converged: 0
KER: 0.003 s
F AND E: 0.264 s
ETOT = -886.31935620, EPOT = -886.48165114, EKIN = 0.05687440, T = 18.33331009, ResErr = 0.000000, t = 1.3 s
ETOT = -886.29036969, EPOT = -886.46293555, EKIN = 0.08531159, T = 27.49996514, ResErr = 0.000000, t = 1.3 s
ETOT = -886.29060339, EPOT = -886.44544501, EKIN = 0.08531159, T = 27.49996514, ResErr = 0.000000, t = 1.3 s
ETOT = -886.29068276, EPOT = -886.45803150, EKIN = 0.08531159, T = 27.49996514, ResErr = 0.000000, t = 1.3 s
0.015497728 GB


########## Step = 1 ##########
   LMAX: tensor([7, 8, 7, 8], device='cuda:0', dtype=torch.int32)
H0: 0.980 s
H1: 0.009 s
  rank: 0, batch 0, Fel = 0.000021
  rank: 0, batch 1, Fel = 0.000021
  rank: 0, batch 2, Fel = 0.000022
  rank: 0, 

# 2. MD without PBC
### Turn on PBC if necessary
## First, SCF and forces

In [9]:
%%time
dftorch_params = {
    "coul_method": "FULL",  # 'FULL' for full coulomb matrix, 'PME' for PME method
    "Coulomb_acc": 5e-5,  # Coulomb accuracy for full coulomb calcs or t_err for PME
    "cutoff": 12.0,  # Coulomb cutoff
    "PME_order": 4,  # Ignored for FULL coulomb method
    "SCF_MAX_ITER": 100,  # Maximum number of SCF iterations
    "SCF_TOL": 1e-6,  # SCF convergence tolerance on density matrix
    "SCF_ALPHA": 0.1,  # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.
    "KRYLOV_MAXRANK": 20,  # Maximum Krylov subspace rank
    "KRYLOV_TOL": 1e-6,  # Krylov subspace convergence tolerance in SCF
    "KRYLOV_TOL_MD": 1e-4,  # Krylov subspace convergence tolerance in MD SCF
    "KRYLOV_START": 10,  # Number of initial SCF iterations before starting Krylov acceleration
}

# Initial data, load atoms and coordinates, etc in COORD.dat
device = "cuda" if torch.cuda.is_available() else "cpu"
filename = "COORD.xyz"  # Solvated acetylacetone and glycine molecules in H20, Na, Cl
# LBox = torch.tensor([45.0, 45.0, 40.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.
LBox = None
# Create constants container. Set path to SKF files.
const = Constants(
    filename,
    "sk_orig/mio-1-1/mio-1-1/",
    magnetic_hubbard_ldep=False,  # atom/shell-resolved spin parameters w for open-shell.
).to(device)

# Create structure object. Define total charge and electronic temperature.
structure1 = Structure(filename, LBox, const, charge=0, Te=600.0, device=device)

# Create ESDriver object and run SCF calculation
# electronic_rcut and repulsive_rcut are in Angstroms.
# They should be >= cutoffs defined in SKF files for the element pair with largest cutoff present in the system.
es_driver = ESDriver(
    dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device
)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const)  # Calculate forces after SCF

coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
### Do _scf ###
  Initial dm_fermi

Starting cycle
Iter 1
Res = 1.909176454, dEc = 1.947740024, t = 0.1 s

Iter 2
Res = 1.378555914, dEc = 0.531737232, t = 0.0 s

Iter 3
Res = 0.994133558, dEc = 0.329875968, t = 0.0 s

Iter 4
Res = 0.716710558, dEc = 0.210435680, t = 0.0 s

Iter 5
Res = 0.516895688, dEc = 0.137885282, t = 0.0 s

Iter 6
Res = 0.373106348, dEc = 0.092571922, t = 0.0 s

Iter 7
Res = 0.269664262, dEc = 0.063480285, t = 0.0 s

Iter 8
Res = 0.195243480, dEc = 0.044317646, t = 0.0 s

Iter 9
Res = 0.141685354, dEc = 0.031404017, t = 0.0 s

Iter 10
Res = 0.103121096, dEc = 0.022529893, t = 0.0 s

Iter 11
  rank: 0, Fel = 0.000984
  rank: 1, Fel = 0.000069
  rank: 2, Fel = 0.000014
  rank: 3, Fel = 0.000001
Res = 0.075331879, dEc = 0.064600423, t = 0.2 s

Iter 12
  rank: 0, Fel = 0.000000
Res = 0.000014816, dEc = 0.000011798, t = 0.0 s

Iter 13
  rank: 0, Fel = 0.000000
Res = 0.000004501, dEc = 0.000000508, t = 0.0 s

Iter 14
  ra

In [10]:
torch.manual_seed(0)
temperature_K = torch.tensor(1.0, device=structure1.device)
mdDriver = MDXL(es_driver, const, temperature_K=temperature_K)
# Set number of steps, time step (fs), dump interval and trajectory filename
mdDriver.run(
    structure1,
    dftorch_params,
    num_steps=5,
    dt=0.3,
    dump_interval=5,
    traj_filename="md_trj.xyz",
)

########## Step = 0 ##########
coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.063 s
H1: 0.095 s
  rank: 0, Fel = 0.000094
KER: 0.139 s
F AND E: 0.002 s
ETOT = -331.68029672, EPOT = -331.80866399, EKIN = 0.00232668, T = 0.89999886, ResErr = 0.000000, t = 0.3 s
0.015364608 GB


########## Step = 1 ##########
coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.051 s
H1: 0.002 s
  rank: 0, Fel = 0.000239
  rank: 1, Fel = 0.000020
KER: 0.003 s
F AND E: 0.001 s
ETOT = -331.68116571, EPOT = -332.16345363, EKIN = 0.12749828, T = 49.31847879, ResErr = 0.001205, t = 0.1 s
0.01536512 GB


########## Step = 2 ##########
coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.051 s
H1: 0.002 s
  rank: 0, Fel = 0.000110
  rank: 1, Fel = 0.000012
KER: 0.003 s
F AND E: 0.001 s
ETOT = -331.68339822, EPOT = -332.70815608, EKIN = 0.48005540, T = 185.69349980, ResErr = 0.002534, t = 0.1 s
0.015365632 GB


########## Step = 3 ##########
coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.052 s

# 3. Open-Shell MD.
### Turn on PBC if necessary
### No batch mode for open-shell
## First, SCF and forces

In [11]:
%%time
dftorch_params = {
    "UNRESTRICTED": True,
    "BROKEN_SYM": True,  # if True, mix 2 % of lumo in homo at initialization
    "coul_method": "!PME",  # 'FULL' for full coulomb matrix, 'PME' for PME method
    "Coulomb_acc": 5e-5,  # Coulomb accuracy for full coulomb calcs or t_err for PME
    "cutoff": 12.0,  # Coulomb cutoff
    "PME_order": 4,  # Ignored for FULL coulomb method
    "SCF_MAX_ITER": 100,  # Maximum number of SCF iterations
    "SCF_TOL": 5e-6,  # SCF convergence tolerance on density matrix
    "SCF_ALPHA": 0.1,  # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.
    "KRYLOV_MAXRANK": 15,  # Maximum Krylov subspace rank
    "KRYLOV_TOL": 9e-6,  # Krylov subspace convergence tolerance in SCF
    "KRYLOV_TOL_MD": 1e-4,  # Krylov subspace convergence tolerance in MD SCF
    "KRYLOV_START": 10,  # Number of initial SCF iterations before starting Krylov acceleration
}

# Initial data, load atoms and coordinates, etc in COORD.dat
device = "cuda" if torch.cuda.is_available() else "cpu"
filename = "O2.xyz"
LBox = None
# LBox = torch.tensor([45.0, 45.0, 40.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.

# Create constants container. Set path to SKF files.
const = Constants(
    filename,
    "sk_orig/mio-1-1/mio-1-1/",
    magnetic_hubbard_ldep=False,
).to(device)

# Create structure object. Define total charge and electronic temperature.
# charge=0 and spin_pol need to be specified for open-shell systems.
# spin_pol is the number of unpaired electrons, not multiplicity.
# Shared chemical potential is used for both spin channels, so spin_pol has no effect but needs to be set anyways.
structure1 = Structure(
    filename,
    LBox,
    const,
    charge=0,
    spin_pol=0,
    os=dftorch_params["UNRESTRICTED"],
    Te=500.0,
    device=device,
)

# Create ESDriver object and run SCF calculation
# electronic_rcut and repulsive_rcut are in Angstroms.
# They should be >= cutoffs defined in SKF files for the element pair with largest cutoff present in the system.
es_driver = ESDriver(
    dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device
)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const)  # Calculate forces after SCF

coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
### Do _scf ###
  Initial dm_fermi

Starting cycle
Iter 1
Res = 0.076646666, dEc = 0.001485381, t = 0.5 s

Iter 2
Res = 0.134471635, dEc = 0.000282222, t = 0.0 s

Iter 3
Res = 0.233454065, dEc = 0.000228600, t = 0.0 s

Iter 4
Res = 0.388443024, dEc = 0.000185166, t = 0.0 s

Iter 5
Res = 0.580101469, dEc = 0.000149985, t = 0.0 s

Iter 6
Res = 0.717176839, dEc = 0.000121487, t = 0.0 s

Iter 7
Res = 0.738018734, dEc = 0.000098405, t = 0.0 s

Iter 8
Res = 0.692595307, dEc = 0.000079708, t = 0.0 s

Iter 9
Res = 0.630998430, dEc = 0.000064563, t = 0.0 s

Iter 10
Res = 0.570071391, dEc = 0.000052296, t = 0.0 s

Iter 11
  rank: 0, Fel = 0.000000
Res = 0.513742896, dEc = 0.000222948, t = 0.4 s

Iter 12
  rank: 0, Fel = 0.000000
Res = 0.000394597, dEc = 0.000000000, t = 0.0 s

Iter 13
zero norm_dr
Res = 0.000000000, dEc = 0.000000000, t = 0.0 s

CPU times: user 3.14 s, sys: 104 ms, total: 3.25 s
Wall time: 2.31 s


In [12]:
structure1.e_tot, structure1.e_spin

(tensor(-176.9905, device='cuda:0'), tensor(-0.7565, device='cuda:0'))

In [13]:
structure1.q_tot_atom

tensor([ 2.6645e-15, -1.4100e-14], device='cuda:0')

In [14]:
# net spin per shell
structure1.net_spin_sr

tensor([-1.1102e-16, -1.0000e+00,  4.4409e-16, -1.0000e+00], device='cuda:0')

In [15]:
# net spin (sums to a number of unpolarized electrons)
structure1.net_spin_sr.sum()

tensor(-2.0000, device='cuda:0')

## MD

In [16]:
torch.manual_seed(0)
temperature_K = torch.tensor(300.0, device=structure1.device)
mdDriver = MDXLOS(es_driver, const, temperature_K=temperature_K)
# mdDriver.NoRank = True
# Set number of steps, time step (fs), dump interval and trajectory filename
mdDriver.run(
    structure1,
    dftorch_params,
    num_steps=5,
    dt=0.2,
    dump_interval=5,
    traj_filename="md_trj.xyz",
)

########## Step = 0 ##########
coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.064 s
H1: 0.405 s
  rank: 0, Fel = 0.000000
KER: 0.343 s
F AND E: 0.816 s
ETOT = -176.95167202, EPOT = -176.98782018, EKIN = 0.03877800, T = 149.99980985, ResErr = 0.000000, t = 1.6 s
0.013203456 GB


########## Step = 1 ##########
coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.050 s
H1: 0.003 s
  rank: 0, Fel = 0.000000
KER: 0.003 s
F AND E: 0.003 s
ETOT = -176.95166954, EPOT = -176.98497813, EKIN = 0.03615064, T = 139.83674359, ResErr = 0.001184, t = 0.1 s
0.013203968 GB


########## Step = 2 ##########
coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.049 s
H1: 0.003 s
  rank: 0, Fel = 0.000000
KER: 0.003 s
F AND E: 0.003 s
ETOT = -176.95166674, EPOT = -176.98196970, EKIN = 0.03331139, T = 128.85403952, ResErr = 0.000171, t = 0.1 s
0.01320448 GB


########## Step = 3 ##########
coulomb_matrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.049 s
H1: 0.003 s
  rank: 0, Fel = 0.000000
KER: 0.003